# Difficulty Level

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

C:\Users\mgopa\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mgopa\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\mgopa\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Step 1: Load and preprocess the dataset
df = pd.read_csv('Data.csv')  
questions = df['Questions'].tolist()
difficulty = df['Difficulty Level'].tolist()

In [3]:
# Step 2: Split the dataset into train and test sets
train_questions, test_questions, train_difficulty, test_difficulty = train_test_split(
    questions, difficulty, test_size=0.2, random_state=42
)

In [4]:
# Step 3: Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

C:\Users\mgopa\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mgopa\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
# Step 4: Tokenize and encode the input sequences
train_encodings = tokenizer(
    train_questions,
    truncation=True,
    padding=True,
    max_length=128,
    add_special_tokens=True,
    return_tensors='pt'
)
test_encodings = tokenizer(
    test_questions,
    truncation=True,
    padding=True,
    max_length=128,
    add_special_tokens=True,
    return_tensors='pt'
)

In [6]:
# Step 5: Create PyTorch datasets
difficulty_mapping = {
    'Easy': 0,
    'Medium': 1,
    'Hard': 2
}

train_labels = torch.tensor([difficulty_mapping[label] for label in train_difficulty])
test_labels = torch.tensor([difficulty_mapping[label] for label in test_difficulty])

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)


In [7]:
# Step 6: Define model and optimizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
# Step 7: Create data loaders
train_sampler = RandomSampler(train_dataset)
test_sampler = SequentialSampler(test_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=16)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=16)

In [9]:
# Step 8: Train the model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [10]:
epochs = 5
for epoch in range(epochs):
    epoch_loss = 0.0
    progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch + 1), leave=False, disable=False)
    for batch in progress_bar:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        epoch_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

    print('Epoch {} training loss: {:.3f}'.format(epoch + 1, epoch_loss / len(train_dataloader)))

Epoch 1 training loss: 0.677


Epoch 2 training loss: 0.378


Epoch 3 training loss: 0.287


Epoch 4 training loss: 0.205


Epoch 5 training loss: 0.129


In [11]:
# Step 9: Evaluate the model
model.eval()
eval_loss = 0.0
predictions = []
for batch in test_dataloader:
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

    eval_loss += loss.item()
    predictions.extend(torch.argmax(logits, dim=1).tolist())

eval_loss = eval_loss / len(test_dataloader)
print('Evaluation loss: {:.3f}'.format(eval_loss))

Evaluation loss: 0.383


In [12]:
# Step 10: Print predictions and actual labels

predicted_difficulty = [list(difficulty_mapping.keys())[p] for p in predictions]
actual_difficulty = [list(difficulty_mapping.keys())[t] for t in test_labels.tolist()]

In [13]:
temp=pd.DataFrame({
    'Actual difficulty':actual_difficulty,
    'Predicted difficulty':predicted_difficulty
})

Predicted difficulty: ['Easy', 'Easy', 'Medium', 'Easy', 'Medium', 'Easy', 'Easy', 'Hard', 'Medium', 'Easy', 'Medium', 'Easy', 'Medium', 'Easy', 'Medium', 'Medium', 'Easy', 'Medium', 'Easy', 'Medium', 'Easy', 'Easy', 'Medium', 'Medium', 'Medium', 'Medium', 'Easy', 'Medium', 'Easy', 'Medium', 'Medium', 'Medium', 'Medium', 'Easy', 'Medium', 'Easy', 'Medium', 'Medium', 'Easy', 'Medium', 'Hard', 'Medium', 'Medium', 'Hard', 'Medium', 'Easy', 'Medium', 'Medium', 'Medium', 'Easy', 'Medium', 'Medium', 'Medium', 'Easy', 'Easy', 'Easy', 'Medium', 'Medium', 'Medium', 'Easy', 'Easy', 'Medium', 'Easy', 'Easy', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Easy', 'Easy', 'Medium', 'Medium', 'Medium', 'Medium', 'Hard', 'Easy', 'Hard', 'Easy', 'Medium', 'Medium', 'Easy', 'Hard', 'Easy', 'Medium', 'Easy', 'Hard', 'Medium', 'Medium', 'Medium', 'Easy', 'Easy', 'Medium', 'Medium', 'Easy', 'Easy', 'Easy', 'Medium', 'Easy', 'Medium', 'Medium', 'Medium', 'Easy', 'Easy', 'Medium', 'Easy', 'Medi

In [14]:
temp.head()

,Actual difficulty,Predicted difficulty
0,Easy,Easy
1,Easy,Easy
2,Medium,Medium
3,Easy,Easy
4,Medium,Medium


In [15]:
# Dismathched items count out of 302
count=0
for i in range(len(actual_difficulty)):
    if actual_difficulty[i]!=predicted_difficulty[i]:
        count+=1
print(count)

115


In [16]:
# Model architecture 
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [17]:
# Model parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total Parameters: ", total_params)

Total Parameters:  109484547


In [18]:
# Model configuration
config = model.config
print("Model Configuration: ", config)

Model Configuration:  BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [19]:
# Model layers
for name, param in model.named_parameters():
    print(name, param.shape)

bert.embeddings.word_embeddings.weight torch.Size([30522, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [20]:
# Model accuracy
from sklearn.metrics import accuracy_score

# Convert predicted labels to taxonomy labels
predicted_labels = [list(difficulty_mapping.keys())[p] for p in predictions]

# Convert actual labels to taxonomy labels
actual_labels = [list(difficulty_mapping.keys())[t] for t in test_labels.tolist()]

# Calculate accuracy
accuracy = accuracy_score(actual_labels, predicted_labels)

print("Accuracy: {:.2f}%".format(accuracy * 100))


Accuracy: 89.46%


# Sample Difficulty Level Question 

In [21]:
sample_question = "Who painted the Mona Lisa?"

encoded_question = tokenizer.encode_plus(
    sample_question,
    truncation=True,
    padding=True,
    max_length=128,
    add_special_tokens=True,
    return_tensors='pt'
)
input_ids = encoded_question['input_ids']
attention_mask = encoded_question['attention_mask']


with torch.no_grad():
    model.eval()
    logits = model(input_ids, attention_mask=attention_mask)

predicted_label = torch.argmax(logits[0]).item()
predicted_difficulty = list(difficulty_mapping.keys())[predicted_label]

print("Predicted difficulty:", predicted_difficulty)


Predicted difficulty: Easy


# Bloom's Taxonomy

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

In [23]:
# Step 1: Load and preprocess the dataset
df = pd.read_csv('Data.csv') 
questions = df['Questions'].tolist()
taxonomy = df["Bloom's Taxonomy"].tolist()

In [24]:
# Step 2: Split the dataset into train and test sets
train_questions, test_questions, train_taxonomy, test_taxonomy = train_test_split(
    questions, taxonomy, test_size=0.2, random_state=42
)

In [25]:
# Step 3: Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [26]:
# Step 4: Tokenize and encode the input sequences
train_encodings = tokenizer(
    train_questions,
    truncation=True,
    padding=True,
    max_length=128,
    add_special_tokens=True,
    return_tensors='pt'
)
test_encodings = tokenizer(
    test_questions,
    truncation=True,
    padding=True,
    max_length=128,
    add_special_tokens=True,
    return_tensors='pt'
)

In [27]:
# Step 5: Create PyTorch datasets
taxonomy_mapping = {
    'Remembering': 0,
    'Understanding': 1,
    'Applying': 2,
    'Analyzing': 3,
    'Evaluating': 4,
    'Creating': 5
}

train_labels = torch.tensor([taxonomy_mapping[label] for label in train_taxonomy])
test_labels = torch.tensor([taxonomy_mapping[label] for label in test_taxonomy])

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)


In [28]:
# Step 6: Define model and optimizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [29]:
# Step 7: Create data loaders
train_sampler = RandomSampler(train_dataset)
test_sampler = SequentialSampler(test_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=16)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=16)

In [30]:
# Step 8: Train the model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [31]:
epochs = 5
for epoch in range(epochs):
    epoch_loss = 0.0
    progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch + 1), leave=False, disable=False)
    for batch in progress_bar:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        epoch_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

    print('Epoch {} training loss: {:.3f}'.format(epoch + 1, epoch_loss / len(train_dataloader)))

Epoch 1 training loss: 0.766


Epoch 2 training loss: 0.357


Epoch 3 training loss: 0.274


Epoch 4 training loss: 0.198


Epoch 5 training loss: 0.121


In [32]:
# Step 9: Evaluate the model
model.eval()
eval_loss = 0.0
predictions = []
for batch in test_dataloader:
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

    eval_loss += loss.item()
    predictions.extend(torch.argmax(logits, dim=1).tolist())

eval_loss = eval_loss / len(test_dataloader)
print('Evaluation loss: {:.3f}'.format(eval_loss))

Evaluation loss: 0.478


In [33]:
# Step 10: Print predictions and actual labels

predicted_taxonomy = [list(taxonomy_mapping.keys())[p] for p in predictions]
actual_taxonomy = [list(taxonomy_mapping.keys())[t] for t in test_labels.tolist()]


In [34]:
temp1=pd.DataFrame({
    'Actual taxonomy':actual_taxonomy,
    'Predicted taxonomy':predicted_taxonomy
})

In [35]:
temp1.head()

,Actual taxonomy,Predicted taxonomy
0,Remembering,Remembering
1,Remembering,Remembering
2,Understanding,Understanding
3,Remembering,Remembering
4,Understanding,Understanding


In [36]:
# Dismathched items count out of 302
count=0
for i in range(len(actual_taxonomy)):
    if actual_taxonomy[i]!=predicted_taxonomy[i]:
        count+=1
print(count)

106


In [37]:
# Model architecture 
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [38]:
# Model parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total Parameters: ", total_params)

Total Parameters:  109486854


In [39]:
# Model configuration
config = model.config
print("Model Configuration: ", config)

Model Configuration:  BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [40]:
# Model layers
for name, param in model.named_parameters():
    print(name, param.shape)

bert.embeddings.word_embeddings.weight torch.Size([30522, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [41]:
# Model accuracy
from sklearn.metrics import accuracy_score

# Convert predicted labels to taxonomy labels
predicted_labels = [list(taxonomy_mapping.keys())[p] for p in predictions]

# Convert actual labels to taxonomy labels
actual_labels = [list(taxonomy_mapping.keys())[t] for t in test_labels.tolist()]

# Calculate accuracy
accuracy = accuracy_score(actual_labels, predicted_labels)

print("Accuracy: {:.2f}%".format(accuracy * 100))


Accuracy: 90.28%


# Sample Bloom's Taxonomy Question

In [42]:
sample_question = "Describe the process of photosynthesis."

encoded_question = tokenizer.encode_plus(
    sample_question,
    truncation=True,
    padding=True,
    max_length=128,
    add_special_tokens=True,
    return_tensors='pt'
)
input_ids = encoded_question['input_ids']
attention_mask = encoded_question['attention_mask']


with torch.no_grad():
    model.eval()
    logits = model(input_ids, attention_mask=attention_mask)

predicted_label = torch.argmax(logits[0]).item()
predicted_taxonomy = list(taxonomy_mapping.keys())[predicted_label]

print("Predicted taxonomy:", predicted_taxonomy)

Predicted taxonomy: Understanding
